# IEOR 242A HW4 Starter Code — Fall 2024

In [44]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
# TODO: Put all dependencies here

In [45]:
# Load in data
rawtrain=pd.read_csv('Letters_train_2024.csv')
rawtest=pd.read_csv('Letters_test_2024.csv')
# TODO: Load in data (after analyzing the dataset, delete any ouputs such as df.inf(), df.head(), et).
# this cell should not output anything

# Question 1 (50 points)

In [46]:
# TODO: Create new variable here
rawtrain['isB']=(rawtrain['letter']=='B').astype(bool)
rawtest['isB']=(rawtest['letter']=='B').astype(bool)

In [47]:
# TODO: Split into X and y
X_train=rawtrain.drop(columns=['letter','isB'])
y_train=rawtrain['isB']
X_test=rawtest.drop(columns=['letter','isB'])
y_test=rawtest['isB']

### Part A: Baseline Model (6 points)

In [48]:
# Q1A code

#TODO: find the most common label
def baselineBModel(x):
    return False
# TODO: calcuate baseline accuracy
baseline_1_acc = (y_test == X_test.apply(baselineBModel, axis=1)).mean()
print(f'Baseline Test Accuracy: {baseline_1_acc:.4f}')

Baseline Test Accuracy: 0.7540


### Part B: Logistic Regression (10 points)

In [49]:
# Q1B code

# TODO: For all questions: Create and train model, then make predictions, then calculate accuracy
# TODO: calcuate logistic accuracy accuracy
from sklearn.linear_model import LogisticRegression
model_1b = LogisticRegression(max_iter=10000) #unnormalized data causes convergence issues
model_1b.fit(X_train, y_train)
model_1b_acc = model_1b.score(X_test, y_test)
print(f'Logistic Regression Test Accuracy: {model_1b_acc:.4f}')

Logistic Regression Test Accuracy: 0.9369


### Part C: AUC (4 point)

In [50]:
# Q1C code
# TODO: Calculate logistic AUC
from sklearn.metrics import roc_auc_score
model_1b_auc = roc_auc_score(y_test, model_1b.predict_proba(X_test)[:,1])
print(f'Logistic Regression Test AUC: {model_1b_auc:.4f}')

Logistic Regression Test AUC: 0.9792


### Part D: Cross-validated CART (10 points)

**Written Answer**: As the dataset is relatively small, I simply iterated through all the values from 0 to 0.1 with an interval of 1/10000 for ccp alpha value (empirically, the optimal value should not exceed 0.1). The value with the highest cross validtaion (cv=5) accuracy is selected by GridSearchCV. (It can be optimized for efficiency using cost_complexity_pruning_path to find 'valid/useful' ccp alpha values).

In [51]:
# Q1D Code
# TODO: calculate best CV accuracy
# TODO: get best ccp_alpha
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
model_1d = DecisionTreeClassifier()
param_grid = {'ccp_alpha': np.linspace(0, 0.1, 1000)} #should be a good range
grid = GridSearchCV(model_1d, param_grid, cv=5)
grid.fit(X_train, y_train)
model_1d_best_ccp_alpha = grid.best_params_['ccp_alpha']
model_1d_acc = grid.score(X_test, y_test)
print(f'CV CART Test Accuracy: {model_1d_acc:.4f}')
print(f'Best ccp_alpha: {model_1d_best_ccp_alpha:.4f}')

CV CART Test Accuracy: 0.9422
Best ccp_alpha: 0.0015


### Part E: Random Forest (10 points)


In [52]:
# Q1E Code
# TODO: calculate random forest accuracy
from sklearn.ensemble import RandomForestClassifier
model_1e = RandomForestClassifier(random_state = 2024) # I'll consider RF as improved bagging and need this random state
model_1e.fit(X_train, y_train)
model_1e_acc = model_1e.score(X_test, y_test)
print(f'Random Forest Test Accuracy: {model_1e_acc:.4f}')

Random Forest Test Accuracy: 0.9711


### Part F: Performance Comparison (10 points)

**Written Answer**: Random Forest did the best. In this case, I would argue that accuracy is more important as interpretability of what makes a "B" a "B" is arguably relatively useless to human beings unless it's under certain special circumstances.

In [53]:
# Q1F Code
# TODO: create df to compare performance
model_df = pd.DataFrame({'Model': ['Baseline', 'Logistic Regression', 'CART', 'Random Forest'],
                         'Accuracy': [baseline_1_acc, model_1b_acc, model_1d_acc, model_1e_acc]})
print(model_df)

                 Model  Accuracy
0             Baseline  0.754011
1  Logistic Regression  0.936898
2                 CART  0.942246
3        Random Forest  0.971123


***
# Question 3 (50 points)

In [54]:
# TODO: Redefine target y
y_train = rawtrain['letter']
y_test = rawtest['letter']

### Part A: Baseline Model (5 points)

In [55]:
# Q2A
# TODO: calculate baseline accuracy
letterModel = y_train.mode()[0]
#print(letterModel) #it's P
baseline_2_acc = (y_test == letterModel).mean()
print(f'Baseline Test Accuracy: {baseline_2_acc:.4f}')

Baseline Test Accuracy: 0.2610


### Part B: LDA (8 points)

In [56]:
# Q2B code
# Calculate LDA accuracy
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model_2b = LinearDiscriminantAnalysis()
model_2b.fit(X_train, y_train)
model_2b_acc = model_2b.score(X_test, y_test)
print(f'LDA Test Accuracy: {model_2b_acc:.4f}')

LDA Test Accuracy: 0.9187


### Part C: Cross-validated CART (10 points)

In [57]:
# Q2C Code
# TODO: calculate CV CART accuracy
model_2c = DecisionTreeClassifier()
grid = GridSearchCV(model_2c, param_grid, cv=5)
grid.fit(X_train, y_train)
model_2c_best_ccp_alpha = grid.best_params_['ccp_alpha']
model_2c_acc = grid.score(X_test, y_test)
print(f'CART Test Accuracy: {model_2c_acc:.4f}')

CART Test Accuracy: 0.9123


### Part D: Vanilla Bagging (8 points)

In [58]:
# Q2D
# TODO: Calculate vanilla bagging accuracy
from sklearn.ensemble import RandomForestClassifier
model_2d = RandomForestClassifier(max_features=len(X_train.columns),random_state = 2024)
model_2d.fit(X_train, y_train)
model_2d_acc = model_2d.score(X_test, y_test)
print(f'No CV Random Forest Test Accuracy: {model_2d_acc:.4f}')

No CV Random Forest Test Accuracy: 0.9401


### Part E: Cross-validated Random Forest (10 points)

Written answer: Same as problem 1, because the dataset is relatively small, I simply iterated through all the values from 1 to n_features for max_feature. The value with the highest cross validtaion (cv=5) accuracy is selected by GridSearchCV.

In [59]:
# Q2E
# TODO: Calculate RF accuracy, use cross-validation to select the max_features
model_2e = RandomForestClassifier(random_state = 2024)
param_grid = {'max_features': np.arange(1, len(X_train.columns)+1)}
grid = GridSearchCV(model_2e, param_grid, cv=5)
grid.fit(X_train, y_train)
model_2e_best_max_features = grid.best_params_['max_features']
model_2e_acc = grid.score(X_test, y_test)
print(f'Best max_features: {model_2e_best_max_features}')
print(f'CV Random Forest Test Accuracy: {model_2e_acc:.4f}')

Best max_features: 4
CV Random Forest Test Accuracy: 0.9658


### Part F: Gradient Boosting Classifier (9 points)

In [60]:
# Q2F
# Calculate boosting accuracy
from sklearn.ensemble import GradientBoostingClassifier
model_2f = GradientBoostingClassifier(n_estimators = 300, max_leaf_nodes = 15, random_state = 2024)
model_2f.fit(X_train, y_train)
model_2f_acc = model_2f.score(X_test, y_test)
print(f'GBC Test Accuracy: {model_2f_acc:.4f}')

GBC Test Accuracy: 0.9701
